In [1]:
# Inspired by https://www.sciencedirect.com/science/article/pii/S0377221723000735
# https://www.sciencedirect.com/science/article/pii/S0167637704000537

In [2]:
import gurobipy as gp

# Create a new model
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02


In [3]:
# Initialize problem parameters
scraps = 3
boxes = 5

set_scraps = range(0, scraps)
set_boxes = range(0, boxes + 1)

# Box distance matrix
d = [[0, 1, 2, 3, 4, 5, 6],
    [1, 0, 1, 2, 3, 4, 5],
    [2, 1, 0, 1, 2, 3, 4],
    [3, 2, 1, 0, 1, 2, 3],
    [4, 3, 2, 1, 0, 1, 2],
    [5, 4, 3, 2, 1, 0, 1],
    [6, 5, 4, 3, 2, 1, 0]]

recipes = [[0, 1], [2, 0], [2, 0]]
set_recipes = range(0, len(recipes))

# Set matrix to 1 if scrap s1 comes anywhere before scrap s2 in recipe r
recipes_precedence_matrix = []
for r in set_recipes:
    dict = {}
    for s1 in recipes[r]:
        for s2 in recipes[r]:
            if s1 == s2: continue
            if recipes[r].index(s1) < recipes[r].index(s2):
                dict[(s1, s2)] = 1
            else:
                dict[(s1, s2)] = 0
    
    recipes_precedence_matrix.append(dict)

recipes_precedence_matrix

[{(0, 1): 1, (1, 0): 0}, {(2, 0): 1, (0, 2): 0}, {(2, 0): 1, (0, 2): 0}]

In [4]:
# Create variables

# Scrap s is assigned to box b
x = {}
for scrap in set_scraps:
    for box in set_boxes:
        x[scrap, box] = m.addVar(vtype='B', name=f"x[{scrap},{box}]")

# Box b is in solution path for recipe r
y = {}
for box in set_boxes:
    for recipe in set_recipes:
        y[box, recipe] = m.addVar(vtype='B', name=f"y[{box},{recipe}]")

# Edge b1 -> b2 is in solution path for recipe r
z = {}
for b1 in set_boxes:
    for b2 in set_boxes:
        if b1 != b2:
            for recipe in set_recipes:
                z[b1, b2, recipe] = m.addVar(vtype='B', name=f"z[{b1},{b2},{recipe}]")

# Box b1 precedes b2 somewhere in solution path for recipe r
p = {}
for b1 in set_boxes:
    for b2 in set_boxes:
        if b1 != b2:
            for recipe in set_recipes:
                p[b1, b2, recipe] = m.addVar(vtype='B', name=f"p[{b1},{b2},{recipe}]")

In [5]:
# Add constraints

# Each scrap is assigned to at least one box
for s in set_scraps:
    m.addConstr(sum(x[s, b] for b in set_boxes[1:]) >= 1)

# Each box is assigned at most one scrap
for b in set_boxes[1:]:
    m.addConstr(sum(x[s, b] for s in set_scraps) == 1)

# Dummy box is empty
m.addConstr(sum(x[s, 0] for s in set_scraps) == 0)

# The number of boxes in a solution for recipe r is equal to the number of scraps required by the recipe + 1
for r in set_recipes:
    m.addConstr(sum(y[b, r] for b in set_boxes) == len(recipes[r]) + 1)

# There is exactly one box b in the solution path for recipe r that corresponds to each scrap s
for r in set_recipes:
    for s in recipes[r]:
        m.addConstr(sum(x[s, b] * y[b, r] for b in set_boxes) == 1)

# There is one incoming edge to each box b in the solution path for recipe r
for r in set_recipes:
    for b2 in set_boxes:
        m.addConstr(sum(z[b1, b2, r] for b1 in set_boxes if b1 != b2) == y[b2, r])

# There is one outgoing edge from each box b in the solution path for recipe r
for r in set_recipes:
    for b1 in set_boxes:
        m.addConstr(sum(z[b1, b2, r] for b2 in set_boxes  if b1 != b2) == y[b1, r])

# Subtour elimination constraints
for r in set_recipes:
    for b1 in set_boxes[1:]:
        for b2 in set_boxes[1:]:
            if b1 != b2:
                m.addConstr(p[b1, b2, r] >= z[b1, b2, r])

for r in set_recipes:
    for b1 in set_boxes[1:]:
        for b2 in set_boxes[1:]:
            if b1 != b2:
                m.addConstr(p[b1, b2, r] + p[b2, b1, r] == y[b1, r] * y[b2, r])

for r in set_recipes:
    for b1 in set_boxes[1:]:
        for b2 in set_boxes[1:]:
            for b3 in set_boxes[1:]:
                if b1 != b2 and b2 != b3 and b1 != b3:
                    m.addConstr(p[b1, b2, r] + p[b2, b3, r] + p[b3, b1, r] <= 2)

# Precedence constraint
# for r in set_recipes:
#     for s1 in recipes[r]:
#         for s2 in recipes[r]:
#             if s1 == s2: continue
#             for b1 in set_boxes[1:]:
#                 for b2 in set_boxes[1:]:
#                     if b1 == b2: continue
#                     m.addConstr(x[s1, b1] * x[s2, b2] * recipes_precedence_matrix[r][(s1, s2)] <= p[b1, b2, r])
        

In [6]:
# Set objective function

objective = gp.quicksum(d[b1][b2] * z[b1, b2, r] for b1 in set_boxes for b2 in set_boxes for r in set_recipes if b1 != b2)

m.setObjective(objective, gp.GRB.MINIMIZE)

In [7]:
# Solve it!
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 PRO 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 288 rows, 216 columns and 927 nonzeros
Model fingerprint: 0x57d3d179
Model has 66 quadratic constraints
Variable types: 0 continuous, 216 integer (216 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 121 rows and 33 columns
Presolve time: 0.00s
Presolved: 383 rows, 243 columns, 1104 nonzeros
Variable types: 0 continuous, 243 integer (243 binary)
Found heuristic solution: objective 24.0000000
Found heuristic solution: objective 14.0000000

Root relaxation: objective 9.600000e+

In [8]:
# Show solution
print(f"Optimal objective value: {m.objVal}")

# Print scrap assignments
for box in set_boxes[1:]:
    for scrap in set_scraps:
        if x[scrap, box].X > 0:
            print(f"Scrap {scrap} is assigned to box {box}")

# Print solution paths
for recipe in set_recipes:
    print(f"Recipe {recipe} solution path:")
    for b1 in set_boxes:
        for b2 in set_boxes:
            if b1 != b2 and z[b1, b2, recipe].X > 0:
                print(f"Box {b1} -> Box {b2}")
    

Optimal objective value: 14.0
Scrap 0 is assigned to box 1
Scrap 2 is assigned to box 2
Scrap 1 is assigned to box 3
Scrap 1 is assigned to box 4
Scrap 1 is assigned to box 5
Recipe 0 solution path:
Box 0 -> Box 1
Box 1 -> Box 3
Box 3 -> Box 0
Recipe 1 solution path:
Box 0 -> Box 1
Box 1 -> Box 2
Box 2 -> Box 0
Recipe 2 solution path:
Box 0 -> Box 1
Box 1 -> Box 2
Box 2 -> Box 0
